In [ ]:
import glob
from keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
import tensorflow as tf
import numpy as np

### Define user parameters and definitions

In [ ]:
# directory containing the trained modelfiles
model_direc = '/direcc/jpflug/ML_layers/WUS_tiles/temp/trial8/'

# in the instance when multiple model types exist, provide wildcard reader to read in correct set
model_prefix = 'linear_bestoutput*.hdf5'

### Script functions

In [ ]:
# ensure this is the same loss function used for the model training
# 1_trainModel.ipynb
def default_mean_squared_error(y_true, y_pred):
    loss = tf.square(y_true - y_pred)
    return tf.reduce_mean(loss)
    
# Register custom loss functions
get_custom_objects().update({
    'default_mean_squared_error': default_mean_squared_error
})

### Read in and prepare the model data for predictions
#### Please review previous scripts for more detail

In [ ]:
# load the ML models
models = sorted(glob.glob(model_direc+model_prefix))

# load the normalization
# order of form: SWEmax,SWEanom_min,SWEanom_max,min_T,max_T,max_P
norm_bounds = np.load(norm_file)

In [ ]:
# read in data using preferred approach 
# Please review 0_prepareData and 1_trainModel for data prep examples
# required data to run the model: dsSCF, SCFaccum, dsT, dsP

### Run the LSTM model

In [ ]:
# loop through the selected models to run
for mod_name in models:
    model = load_model(mod_name,
                       custom_objects={'output_mse1': default_mean_squared_error, 'output_mse3': default_mean_squared_error})
    swe = model.predict([dsSCF,SCFaccum,dsT,dsP])
    # convert from normalized space
    # default LSTM output
    swe[0] = swe[0].T*norm_bounds[0]
    # melt-corrected LSTM output
    swe[1] = swe[1].T*norm_bounds[0]